In [1]:
import pandas as pd


In [10]:
pattern = r"^[A-Z][0-9]{4}|[0-9]{5}|[0-9]{4}[A-Z]$"

def payer_category(x):
    payers = {
        'Charge': 'gross',
        'Self Pay Price': 'cash',
        'Min Negotiated Rate': 'min',
        'Max Negotiated Rate': 'max',
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [3]:
file = '540519577_mary-washington-hospital_standardcharges.xlsx'
df = pd.read_excel('.\\input_files\\' + file)
df1 = df

In [21]:
df = df1

In [22]:
df.rename(columns={
    'Code': 'code',
    'Desc.': 'description'
}, inplace=True)

In [23]:
cols = df.columns.tolist()
id_vars = ['code', 'description']
value_vars = cols[3:]
value_vars.append('Charge')

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name= 'standard_charge')

In [24]:
df.dropna(subset=['standard_charge'], inplace=True)

In [25]:
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']

In [26]:
df['payer_category'] = df['payer'].apply(payer_category)

In [27]:
ccn = {
        '540519577': '490022',
        '134316364': '490140'
}

ein = file.split('_')[0]
df['hospital_id'] = ccn[ein]

# df['hcpcs_cpt'].fillna('""', inplace=True)
# df['code'].fillna('""', inplace=True)

df['hcpcs_cpt'] = df['hcpcs_cpt'].astype(str)
# df.loc[df['hcpcs_cpt'].str.contains(pattern), 'hcpcs_cpt'] = '""'
mask = ~df['hcpcs_cpt'].str.contains(pattern)
df.loc[mask, 'hcpcs_cpt'] = '' # '""'
df.loc[df['hcpcs_cpt'].str.contains('-'), 'hcpcs_cpt'] = pd.NA # '""'

df['code'] = df['code'].str.strip()
df['code'] = df['code'].apply(lambda x: str(x).zfill(3) if len(str(x)) < 3 else x)

df['apr_drg'] = ''
df.loc[df['code'].str.contains('-') & ~df['code'].isna(), 'apr_drg'] = df['code']

df['ms_drg'] = ''
df.loc[df['code'].str.len() == 3, 'ms_drg'] = df['code']

df['code'].fillna('', inplace=True)


In [30]:
columnsList = ['code', 'hcpcs_cpt', 'apr_drg', 'ms_drg']
for col in columnsList:
    df[col] =  df[col].apply(lambda x: '"' + str(x) + '"' if x != 'nan' else '""')

In [31]:
df

,code,description,payer,standard_charge,hcpcs_cpt,payer_category,hospital_id,apr_drg,ms_drg
15,"""0003M""",Liver dis 10 assays w/nash,Self Pay Price,755.0,"""0003M""",cash,490022,"""""",""""""
30,"""""",HC RM AND BED MED SURG,Self Pay Price,625.0,"""""",cash,490022,"""""",""""""
32,"""""",HC RM AND BED PALLIATIVE CARE,Self Pay Price,483.0,"""""",cash,490022,"""""",""""""
34,"""""",HC RM AND BED RESPITE CARE,Self Pay Price,483.0,"""""",cash,490022,"""""",""""""
36,"""""",HC RM AND BED OBSTETRICS,Self Pay Price,625.0,"""""",cash,490022,"""""",""""""
...,...,...,...,...,...,...,...,...,...
865251,"""U0001""",2019-ncov diagnostic p,Charge,72.0,"""U0001""",gross,490022,"""""",""""""
865252,"""U0002""",Covid-19 lab test non-cdc,Charge,103.0,"""U0002""",gross,490022,"""""",""""""
865253,"""U0003""",Cov-19 amp prb hgh thruput,Charge,200.0,"""U0003""",gross,490022,"""""",""""""
865254,"""U0004""",Cov-19 test non-cdc hgh thru,Charge,200.0,"""U0004""",gross,490022,"""""",""""""


In [ ]:
df.to_csv(file.split('_')[0] + '.csv', index=False)